In [ ]:
using LIBLINEAR, SparseArrays, ProgressMeter, LinearAlgebra

# Solvers

In [ ]:
function mengdi3(A,b,maxiters,C,M)
    n = size(A)[1]
    m = Int(size(A)[2])
    w = zeros(m)
    ξ = spzeros(n)
    for i = 1:maxiters
        α = 1/(i+10)
        w -= α*w
        #ξ -= α*ξ/C
        
        I = zeros(M)
        for s = 1:M
            I[s] = Int(ceil(rand()*n))
        end
        for r = 1:M
            z = zeros(M)
            
            for s = 1:M
                j = Int(I[s])
                a = A[j,:]
                θ = ((a' * w)[1] - 1 + ξ[j])/(sum(a.^2))
                c = min(θ,z[s])
                w -= c .* a
                z[s] -= c
                if z[s] < 1e-15
                    z[s] = 0
                end
            end
        end
        
        if i%100 == 0
            accVal = sum(yval .== sign.(Xval*w))/length(yval)
            acc = sum(y .== sign.(X*w))/length(y)
            @show((acc,accVal))
            flush(stdout)
        end
        
    end
    
    return w
end

In [ ]:
function mengdi(A,b,maxiters,C,M)
    n = size(A)[1]
    m = Int(size(A)[2])
    w = rand(m)
    ξ = spzeros(n)
    for i = 1:maxiters
        α = 1/(i+10)
        w̃ = w - α*w
        #ξ -= α*ξ/C
        
        w′ = copy(w̃) 
        θmax = 0
        jmax = 0
        max = 0
        for s = 1:M
            j = Int(ceil(rand()*n))
            a = A[j,:]
            θ = ((a' * w̃)[1] + ξ[j])/(sum(a.^2))
            if θ < 0
                ŵ = w - θ .* a
                if norm(ŵ-w̃) > max
                    max = norm(ŵ-w̃)
                    w′ = ŵ
                    θmax = θ
                    jmax = j
                end
            end
        end
        
        w = w′
        #ξ[jmax] -= θmax/C
        
        if i%100 == 0
            accVal = sum(yval .== sign.(Xval*w))/length(yval)
            acc = sum(y .== sign.(X*w))/length(y)
            @show((acc,accVal))
            flush(stdout)
        end
        
    end
    
    return w
end

In [ ]:
function SVM(A,b,maxiters,tol,C)
    n = size(A)[1]
    m = Int(size(A)[2])
    z = spzeros(n)
    z′ = spzeros(n)
    w = zeros(m)
    ξ = spzeros(n)
    maxD = tol + 1
    count = 0
    A = b.* A
    for i = 1:maxiters
        j = Int(ceil(rand()*n))
        a = A[j,:]
        θ = ((a' * w)[1] - 1 + ξ[j])/(sum(a.^2)+1/C)
        c = min(θ,z[j])
        w -= c .* a
        ξ[j] -= c/C
        z[j] -= c
        if z[j] < 1e-15
            z[j] = 0
        end
        if abs(c) > maxD
            maxD = abs(c)
        end
            
        c = min(z′[j],ξ[j])
        ξ[j] -= c
        z′[j] -= c
        
    end

    return w
end       

# Generate Data

In [ ]:
N = 10000000
d = 100
X = 1*randn(N,d)
Xval = 1*randn(N,d)
w = randn(d)
y = sign.(X*w)
yval = sign.(Xval*w)
X += randn(N,d);
Xval += randn(N,d);

In [ ]:
sum(abs.(y - sign.(X*w)))/N

In [ ]:
Xnew = X'

# Use LibLinear

setting solver_type=Cint(2) switches to the primal solver. Removing this defaults to the dual solver. 

In [ ]:
@time model = linear_train(y, X', eps=0.001, C=1000, verbose=true, solver_type=Cint(2));

In [ ]:
(predicted_labels, decision_values) = linear_predict(model, Xval');
sum((predicted_labels .== yval))/length(yval)

# Using Project and Forget

In [ ]:
@time w2 = SVM(X,y,10000,0.1,1000)
accVal = sum(yval .== sign.(Xval*w2))/length(yval)
acc = sum(y .== sign.(X*w2))/length(y)
@show((acc,accVal))
flush(stdout)

# Using Mengdi algorithms

In [ ]:
@time mengdi(X,y,10000000,1000,100)